In [1]:
 import random
 import hashlib

In [3]:
def gcd(a,b): 
    if b==0: 
        return a 
    else: 
        return gcd(b,a%b)

In [5]:
 def isPrime(n) : 
    # Corner cases 
    if (n <= 1) : 
        return False
    if (n <= 3) : 
        return True
   
    # This is checked so that we can skip  
    # middle five numbers in below loop 
    if (n % 2 == 0 or n % 3 == 0) : 
        return False
   
    i = 5
    while(i * i <= n) : 
        if (n % i == 0 or n % (i + 2) == 0) : 
            return False
        i = i + 6
   
    return True

In [4]:
 # Get a prime number
 def generatePrime(num = 100):
    L1 = []
    for i in range(60, num + 1): 
        if isPrime(i):
            L1.append(i)
    
    p = random.choice(L1)
    L1.pop(L1.index(p))
    q = random.choice(L1)
    
    t = (p-1)*(q-1)
    n = p*q
    
    for e in range(2,t): 
        if gcd(e,t)== 1: 
            break
    for i in range(1,10): 
        x = 1 + i*t 
        if x % e == 0: 
            d = int(x/e) 
            break
    
    return e,d,n

In [11]:
import random
import math

def isPrime(num):
    if num < 2:
        return False
    for i in range(2, int(math.sqrt(num)) + 1):
        if num % i == 0:
            return False
    return True

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def generatePrime(num=100):
    L1 = [i for i in range(60, num + 1) if isPrime(i)]
    p = random.choice(L1)
    L1.remove(p)
    q = random.choice(L1)
    t = (p - 1) * (q - 1)
    n = p * q
    for e in range(2, t):
        if gcd(e, t) == 1:
            break
    for i in range(1, 10):
        x = 1 + i * t
        if x % e == 0:
            d = x // e
            break
    return e, d, n


In [21]:
import socket, sys, threading

# Generate Keys
e, d, n = generatePrime()
msgFromClient = "Hello UDP Server!. My public Key is: " + str(e) + " " + str(n)
bytesToSend = str.encode(msgFromClient)

# Define Client IP and Port
localIP = "127.0.0.2"
localPort = 3002
serverAddressPort = ("127.0.0.2", 3001)  # Ensure it matches server's port
bufferSize = 1024

# Create UDP socket & Bind
UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
UDPClientSocket.bind((localIP, localPort))  # Bind the client to port 3002

def recv(E, N):
    while True:
        try:
            recieve = UDPClientSocket.recvfrom(bufferSize)
            msg = recieve[0].decode('utf-8')
            token = msg[-64:]
            cipherText = msg[:-64]
            plainText = decrypt(cipherText, d, n)

            tk = hashlib.md5(cipherText.encode()).hexdigest().upper()
            tk1 = decrypt(token, E, N)

            if tk.upper() == tk1.upper():
                if not recieve[0]: sys.exit(0)
                print(plainText)
        except ConnectionResetError:
            print("Connection lost. Exiting...")
            sys.exit(0)

def Send(sA):
    while True:
        message = input("Enter your reply  ").upper()
        try:
            L1 = list(message.split())
            if len(L1) != 3:
                raise Exception('Error')
        except:
            print('You must provide all three input in this sequence: Message , eKey, N')
            sys.exit(0)

        cipherText = convertText(L1[0], int(L1[1]), int(L1[2]))
        m = hashlib.md5(cipherText.encode()).hexdigest().upper()
        N1 = convertText(m, d, n)
        ct = cipherText + N1

        UDPClientSocket.sendto(ct.encode('utf-8'), sA)
        if L1[0] == 'BYE':
            sys.exit(0)

# Send initial message to Server
for _ in range(3):  # Retry up to 3 times
    try:
        UDPClientSocket.sendto(bytesToSend, serverAddressPort)
        bytesAddressPair = UDPClientSocket.recvfrom(bufferSize)
        break
    except ConnectionResetError:
        print("Retrying...")
        time.sleep(1)

# Extract Server's Public Key
message = bytesAddressPair[0].decode('utf-8')
l1 = list(message.split())
E = int(l1[-2])
N = int(l1[-1])
print(E, N)

address = bytesAddressPair[1]
print("Message from Server:", message)
print("Server IP Address:", address)

# Start Threads for Receiving and Sending Messages
threading.Thread(target=recv, args=(E, N)).start()
threading.Thread(target=Send, args=(address,)).start()


5 5963
Message from Server: Hello UDP Client!. My Public key is: 5 5963
Server IP Address: ('127.0.0.2', 3001)


Enter your reply   Hii I am Nice
